In [16]:
from numpy import *
from matplotlib.pyplot import *
from ipywidgets import *

def airfoil_cp_split(r=1.5, delta=2.0, k=1.5, a=0.25, b=0.5,
                     V_inf=10.0, show_cp=True, show_data=False,show_point = False, show_all =False):
    
    # Air @ 15degC properties
    rho = 1.225  # kg/m³
    mu = 1.81e-5  # kg/m·s

    # Circle discretization
    theta = linspace(0, 2*pi, 1000)
    z = (r * cos(theta) + a) + 1j * (r * sin(theta) + b)

    # Kármán–Trefftz parameters
    n = 2 - delta / pi
    z_plus_k = z + k
    z_minus_k = z - k

    # Airfoil mapping
    f_z = k * n * (z_plus_k**n + z_minus_k**n) / (z_plus_k**n - z_minus_k**n)
    x1, y1 = real(f_z), imag(f_z)

    # Derivative of conformal map for velocity
    dzdzeta = (2 * k * n * z_plus_k**(n - 1) * z_minus_k**(n - 1) *
               (z_minus_k**n + z_plus_k**n)) / (z_plus_k**n - z_minus_k**n)**2
    V = abs(1 / dzdzeta) * V_inf
    Cp = 1 - (V / V_inf)**2

    # Compute chord length and Reynolds number
    chord = max(x1) - min(x1)
    Re = rho * V_inf * chord / mu

    # Split upper (theta: 0 to pi) and lower (theta: pi to 2pi)
    upper_idx = where((theta >= 0) & (theta <= pi))[0]
    lower_idx = where((theta > pi) & (theta <= 2*pi))[0]

    x_upper, Cp_upper = x1[upper_idx], Cp[upper_idx]
    x_lower, Cp_lower = x1[lower_idx], Cp[lower_idx]

    # Sort both upper and lower by x for smooth plots
    upper_sort = argsort(x_upper)
    lower_sort = argsort(x_lower)
    x_upper_sorted = x_upper[upper_sort]
    Cp_upper_sorted = Cp_upper[upper_sort]
    x_lower_sorted = x_lower[lower_sort]
    Cp_lower_sorted = Cp_lower[lower_sort]

    # Plotting
    figure(figsize=(12, 6))

    if show_cp:
        # Cp vs x (upper and lower)
        subplot(1, 2, 1)
        plot(x_upper_sorted, Cp_upper_sorted, 'b-', label='Upper Surface')
        plot(x_lower_sorted, Cp_lower_sorted, 'r-', label='Lower Surface')
        xlabel("x (Chord Position)")
        ylabel("Pressure Coefficient $C_p$")
        title(f"$C_p$ Distribution\nRe = {Re:.2e}, Chord = {chord:.4f} m")
        gca().invert_yaxis()
        legend()
        grid(True)

    # Airfoil shape
    subplot(1, 2, 2)
    plot(x1, y1, 'k-', label='Airfoil Surface')
    scatter(x1[::100], y1[::100], c=Cp[::100], cmap='coolwarm', edgecolors='black')
    colorbar(label="$C_p$")
    xlabel('x')
    ylabel('y')
    axis('equal')
    title("Airfoil Shape with $C_p$")
    grid(True)

    tight_layout()
    show()

    #show points script
    if show_point:
        print("\nJoukowsky Transformed Coordinates (x1, y1):")
        if show_all:
            for xi, yi in zip(x1, y1):
                print(f"{xi:.4f}, {yi:.4f}")
        else:
            for i in range(10):
                print(f"{x1[i]:.4f}, {y1[i]:.4f}")

    if show_data:
        print(f"Chord Length: c = {chord:.4f}")
        print(f" /t Reynolds Number: Re = {Re:.2e}")
        print("\nSample Upper Surface Cp:")
        for i in range(5):
            print(f"x = {x_upper_sorted[i]:.4f}, Cp = {Cp_upper_sorted[i]:.4f}")
        print("Sample Lower Surface Cp:")
        for i in range(5):
            print(f"x = {x_lower_sorted[i]:.4f}, Cp = {Cp_lower_sorted[i]:.4f}")

In [19]:
# Interactive GUI
interact(airfoil_cp_split,
         r=FloatSlider(value=1.5, min=-13, max=13.0, step=0.1, description='Radius (r):'),
         delta=FloatSlider(value=2.0, min=-13, max=13.0, step=0.1, description='Delta (δ):'),
         k=FloatSlider(value=1.5, min=0.1, max=10.0, step=0.1, description='Constant (k):'),
         a=FloatSlider(value=0.25, min=-2.0, max=2.0, step=0.05, description='Origin x (a):'),
         b=FloatSlider(value=0.5, min=-2.0, max=2.0, step=0.05, description='Origin y (b):'),
         V_inf=FloatSlider(value=10.0, min=1.0, max=100.0, step=1.0, description='Freestream Velocity $V_∞$ [m/s]'),
        show_point=Checkbox(value=True, description='Show Coordinates'),
         show_all=Checkbox(value=False, description='Show All (1000 pts)'),
         show_cp=Checkbox(value=True, description='Show $C_p$ Plot'),
         show_data=Checkbox(value=False, description='Print Cp Values'));

interactive(children=(FloatSlider(value=1.5, description='Radius (r):', max=13.0, min=-13.0), FloatSlider(valu…